# Understanding Winton Stock Market Data

과거 몇일전의 데이터를 받아서 몇일후에 주가를 예측하는 Challenge입니다.<br>
5일동안의 데이터를 받습니다. D-2, D-1, D, D+1, D+2 중에서 D-2, D-1, 그리고 D의 일부 시간 데이터를 받을뒤 남은 D, D+1, D+2의 주가를 예측하면 됩니다.<br>

D날에는 


<img src="images/winton-stock-data.png">

### **train.csv**
   * Feature_1 - Feature_25
   * Ret_MinusTwo, Ret_MinusOne
   * Ret_2 - Ret_120
   * Ret_121 - Ret_180: target variables
   * Ret_PlusOne, Ret_PlusTwo: target variables
   * Weight_Intraday, Weight_Daily
   
### **test.csv**
   * Feature_1 - Feature_25
   * Ret_MinusTwo, Ret_MinusOne
   * Ret_2 - Ret_120
   

### Data Fields



* **Feature_1 ~ Feature25:** prediction에 관련된 여러가지 features
* **Ret_MinusTwo:** D-2의 폐장시점부터 D1의 폐장시점까지의 수익률입니다. (1 day)
* **Ret_MinusOne:** D-1의 폐장시점부터 D의 일중 변동(Intraday returns)전까지의 수익률입니다. (대략 1/2 day)
* **Ret_2 ~ Ret_120:** D의 특정시점의 대략적인 분단위 수익률입니다. Ret_2의 경우 t=1 과 t=2사이의 수익률입니다.
* **Ret_121 ~ Ret_180:** D의 특정시점의 대략적인 분단위 수익률입니다. **target variables로서 {id}_{1~60}으로 예측을 해야하는 부분입니다**
* **Ret_PlusOne:** Ret_180이 계산 종료된 시점부터 D+1의 폐장시점까지의 수익률입니다. (대략 1 day). **target variables로서 {id}_61 로 예측을 해야 합니다.**
* **Ret_PlusTwo:** D+1의 폐장시점부터 D+2의 폐장시점까지의 수익률입니다. (1 day) **target variables로서 {id}_62로 예측을 해야 합니다.**
* **Weight_Intraday:** Ret_121 부터 Ret_180 까지의 일중 수익률을 계산시 사용한 가중치(weight)입니다.
* **Weight_Daily:**: Ret_PlusOne 그리고 Ret_PlusTwo를 예측할때 사용한 가중치 (Weight)입니다.

### Evaluation

* https://www.kaggle.com/c/the-winton-stock-market-challenge#evaluation

Winston Stock Market Challenge에서는 [Weighted Mean Absolute Error](https://www.kaggle.com/wiki/WeightedMeanAbsoluteError) 를 사용해서 평가를 합니다. 

$$ WMAE = \frac{1}{n}\sum\limits_{i=1}^{n} w_i \cdot \left|y_i - \hat{y_i}\right| $$

즉 여기서 $ w_i $는 Weight_Intraday 그리고 Weight_Daily 를 가르킵니다.

In [10]:
%pylab inline
import numpy as np
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Loading Data

In [2]:
train = pd.read_csv('/dataset/winton-stock-market-challenge/train.csv')

In [3]:
train[['Id', 'Feature_1', 'Feature_25', 
       'Ret_MinusTwo', 'Ret_MinusOne', 
       'Ret_2', 'Ret_180', 
       'Ret_PlusOne', 'Ret_PlusTwo', 
       'Weight_Intraday', 'Weight_Daily']].head()

,Id,Feature_1,Feature_25,Ret_MinusTwo,Ret_MinusOne,Ret_2,Ret_180,Ret_PlusOne,Ret_PlusTwo,Weight_Intraday,Weight_Daily
0,1,NaN,NaN,0.055275,-0.010770,0.000003,-0.001974,-0.019512,0.028846,1.251508e+06,1.564385e+06
1,2,NaN,-0.709462,0.009748,0.002987,-0.000487,0.000027,-0.002939,-0.010253,1.733950e+06,2.167438e+06
2,3,NaN,-1.019370,0.003077,0.006181,-0.000782,0.000784,-0.024791,0.015711,1.529197e+06,1.911497e+06
3,4,NaN,NaN,0.000984,0.014106,0.000277,0.000341,-0.005680,-0.002190,1.711569e+06,2.139462e+06
4,5,6.0,3.219820,-0.018224,0.011065,-0.001232,-0.000004,0.036104,-0.026552,1.267270e+06,1.584088e+06


## Feature Summary

In [12]:
features = train[[i for i in range(1, 26)]]
features.describe()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25
count,6687.000000,30854.000000,38763.000000,32279.000000,40000.000000,38067.000000,40000.000000,39531.000000,38125.000000,20529.000000,...,39390.000000,39354.000000,39432.000000,38810.000000,32174.000000,38982.000000,38655.000000,38289.000000,39274.000000,39345.000000
mean,3.590250,-0.117558,0.558392,0.405572,5.482775,0.430972,49244.971525,0.196958,10.680289,4.744703,...,1.007362,-0.549725,0.803059,-1.205438,5.267359,0.605593,-0.773089,0.799833,-1.209290,-0.329675
std,2.798532,1.236250,0.902233,0.799082,2.942324,1.498274,28242.409717,0.138485,2.850634,0.865096,...,0.085488,0.936833,1.165442,0.642426,2.549227,1.319158,1.389229,1.288040,1.739656,0.958661
min,1.000000,-3.440521,-4.643526,-5.440596,1.000000,-0.936644,338.000000,0.009800,0.000000,1.000000,...,1.000000,-2.613987,-5.758047,-3.292909,2.000000,-1.514998,-5.819912,-7.221387,-11.442205,-1.903876
25%,1.000000,-0.967186,-0.110192,-0.111696,2.000000,-0.265555,26143.000000,0.016600,9.000000,5.000000,...,1.000000,-1.021216,0.057598,-1.619718,3.000000,-0.294925,-1.787615,0.539979,-1.838688,-0.830749
50%,3.000000,-0.389162,0.437228,0.403516,6.000000,0.055564,48457.000000,0.213800,11.000000,5.000000,...,1.000000,-0.599050,0.587005,-1.169327,5.000000,0.308468,-0.699112,0.962580,-0.868435,-0.551550
75%,6.000000,0.414442,1.064754,0.945944,8.000000,0.559921,72387.000000,0.331800,12.000000,5.000000,...,1.000000,-0.184854,1.321231,-0.735786,7.000000,1.109743,0.282958,1.415303,-0.129465,-0.257543
max,10.000000,4.175150,4.530405,2.953163,10.000000,12.609885,99861.000000,0.365000,36.000000,6.000000,...,2.000000,7.683857,6.352352,0.898236,10.000000,7.737020,2.284991,3.228906,2.526654,4.020332


## Return Summary

In [47]:
returns = train[[26, 27, 28, 146, 147, 206, 207, 208]]
returns.describe()

,Ret_MinusTwo,Ret_MinusOne,Ret_2,Ret_120,Ret_121,Ret_180,Ret_PlusOne,Ret_PlusTwo
count,40000.000000,40000.000000,3.894600e+04,4.000000e+04,4.000000e+04,40000.000000,40000.000000,40000.000000
mean,0.000784,-0.000803,5.600930e-06,-2.081735e-06,-3.539200e-05,-0.000057,-0.000210,0.000012
std,0.028279,0.030569,9.501528e-04,1.207381e-03,1.095917e-03,0.001270,0.025039,0.024160
min,-0.536283,-0.514720,-1.311777e-02,-2.844145e-02,-2.174075e-02,-0.058046,-0.627690,-0.450779
25%,-0.010687,-0.010830,-3.215495e-04,-3.465326e-04,-3.439227e-04,-0.000413,-0.010521,-0.010055
50%,0.000112,-0.000665,-1.989567e-07,-8.172886e-08,7.410876e-08,-0.000002,-0.000258,-0.000258
75%,0.010987,0.008976,3.122045e-04,3.437908e-04,3.389849e-04,0.000364,0.010005,0.009772
max,0.894024,0.852139,3.214902e-02,1.160767e-01,5.115931e-02,0.026112,0.795602,0.303038
